In [4]:
pip install xmlschema

StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.8/228.8 kB 91.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
import xmlschema

spark = SparkSession.builder \
    .appName("Lakehouse XML XSD") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.12.0") \
    .getOrCreate()

# Step 1: Parse the XSD file to define the table structure
xsd_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/casino_data_schema.xsd"  # Adjust to your XSD file path
xml_path = "/lakehouse/default/Files/casino_data.xml"  # Adjust to your XML file path

# Load the XSD schema
xsd_schema = xmlschema.XMLSchema(xsd_path)

# Function to map XSD types to PySpark types
def get_spark_type(xsd_type):
    if xsd_type == "string":
        return StringType()
    elif xsd_type == "int":
        return IntegerType()
    elif xsd_type == "decimal":
        return DecimalType(10, 2)  # You can adjust precision and scale
    # Add more mappings as needed
    return StringType()  # Default fallback

# Extract schema details from XSD
fields = []
for element in xsd_schema.elements.values():
    field_name = element.name
    data_type = element.type.local_name
    fields.append(StructField(field_name, get_spark_type(data_type), True))

# Create the DataFrame schema
schema = StructType(fields)

# Step 2: Read XML data using the generated schema
df = spark.read \
    .format("xml") \
    .schema(schema) \
    .load(xml_path)

# Step 3: Show the DataFrame
df.show()


StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 15, Finished, Available, Finished)

URLError: <urlopen error unknown url type: abfss>

In [43]:
from pyspark.sql import SparkSession
import xmlschema
import os
import xml.etree.ElementTree as ET
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, TimestampType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Lakehouse XML XSD Reader") \
    .getOrCreate()

# Set your ABFS path
abfs_xsd_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/casino_data_schema.xsd"  # Update with your actual path

# Step 1: Read the XSD file into memory
xsd_df = spark.read.text(abfs_xsd_path)

# Write the XSD content to a local temporary file
local_xsd_path = "/tmp/casino_data_schema.xsd"  # Temporary local path
xsd_content = "\n".join(row.value for row in xsd_df.collect())

# Save to local path
with open(local_xsd_path, "w") as f:
    f.write(xsd_content)

# Step 2: Load the XSD schema using xmlschema
xsd_schema = xmlschema.XMLSchema(local_xsd_path)

# Function to parse XSD and create a schema
def parse_xsd(xsd_file):
    tree = ET.parse(xsd_file)
    root = tree.getroot()
    schema = {}
    
    # Iterate through elements in XSD
    for complex_type in root.findall('.//xs:complexType', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
        type_name = complex_type.get('name')
        fields = []
        
        for sequence in complex_type.findall('.//xs:sequence', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
            for element in sequence.findall('xs:element', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
                field_name = element.get('name')
                field_type = element.get('type')
                # Map XSD types to Spark types
                spark_type = None
                if field_type == 'xs:string':
                    spark_type = StringType()
                elif field_type == 'xs:int':
                    spark_type = IntegerType()
                elif field_type == 'xs:decimal':
                    spark_type = DecimalType()
                elif field_type == 'xs:dateTime':
                    spark_type = TimestampType()
                
                if spark_type is not None:
                    fields.append(StructField(field_name, spark_type, True))
        
        schema[type_name] = StructType(fields)
    
    return schema

# Extract row tags
def extract_row_tags(xsd_path):
    # Parse the XSD file
    schema_tree = ET.parse(xsd_path)
    root = schema_tree.getroot()

    # Find all complex types with sequence elements
    row_tags = []
    for complex_type in root.findall('.//xs:complexType', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
        for sequence in complex_type.findall('./xs:sequence', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
            for element in sequence.findall('xs:element', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
                row_tags.append(element.attrib['name'])

    return row_tags

row_tags = extract_row_tags(local_xsd_path)

# Load the XML data
xml_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/casino_data.xml"  # Adjust to your XML file path

# Read XML data for each row tag dynamically
schema = parse_xsd(local_xsd_path)

# Dictionary to store DataFrames for each row tag
dataframes = {}

for tag in row_tags:
    # Read the XML file for the current row tag
    df = spark.read \
        .format("xml") \
        .option("rowTag", tag) \
        .load(xml_path)

    # Dynamically select columns based on parsed schema
    if tag in schema:
        df = df.select(*[f"{tag}.{field.name}" for field in schema[tag].fields])
    
    # Store the DataFrame in the dictionary
    dataframes[tag] = df
    
    # Create a temporary table/view for SQL queries
    df.createOrReplaceTempView(tag)
    
    print(f"Data for {tag}:")
    #df.show(truncate=False)

print('new line')
print(dataframes)
#for table_name, df in dataframes.items():
 #   if df is not None and not df.isEmpty():  # Check if the DataFrame is not empty
        # Write the DataFrame to the Lakehouse as a Parquet file
 #       df.write.mode("overwrite").format("delta").saveAsTable(table_name)
 #   else:
 #       print(f"DataFrame for {table_name} is empty or None, skipping...")
# Clean up the local XSD file if needed
os.remove(local_xsd_path)
for tag in row_tags:
    print(f"Processing table for {tag}:")

    # Dynamically construct table name
    table_name = f"{tag}_table"  # Example: for tag 'user', table_name becomes 'user_table'
    
    # Query the temporary view created for the tag
    temp_df = spark.sql(f"SELECT * FROM {tag}")

    # Check if the DataFrame is not empty
    if temp_df is not None and not temp_df.isEmpty():
        # Write the DataFrame to the Lakehouse in Delta format
        temp_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable(table_name)

        # Display the content of the saved table (optional)
        print(f"Table {table_name} data:")
        spark.sql(f"SELECT * FROM {table_name}").show(truncate=False)
    else:
        print(f"DataFrame for {table_name} is empty or None, skipping...")



StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 46, Finished, Available, Finished)

Data for users:
Data for games:
Data for transactions:
Data for user:
Data for user_id:
Data for username:
Data for email:
Data for password_hash:
Data for balance:
Data for registration_date:
Data for status:
Data for game:
Data for game_id:
Data for name:
Data for type:
Data for min_bet:
Data for max_bet:
Data for provider:
Data for transaction:
Data for transaction_id:
Data for user_id:
Data for amount:
Data for transaction_type:
Data for transaction_date:
Data for game_id:
Data for status:
new line
{'users': DataFrame[user: array<struct<balance:double,email:string,password_hash:string,registration_date:timestamp,status:string,user_id:bigint,username:string>>], 'games': DataFrame[game: array<struct<game_id:bigint,max_bet:double,min_bet:double,name:string,provider:string,type:string>>], 'transactions': DataFrame[transaction: array<struct<amount:double,game_id:bigint,status:string,transaction_date:timestamp,transaction_id:bigint,transaction_type:string,user_id:bigint>>], 'user': DataF

AnalysisException: Data used in creating the Delta table doesn't have any columns.

In [55]:
from pyspark.sql import SparkSession
import xmlschema
import os
import xml.etree.ElementTree as ET
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, TimestampType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Lakehouse XML XSD Reader") \
    .getOrCreate()

# Set your ABFS path
abfs_xsd_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/casino_data_schema.xsd"  # Update with your actual path

# Step 1: Read the XSD file into memory
xsd_df = spark.read.text(abfs_xsd_path)

# Write the XSD content to a local temporary file
local_xsd_path = "/tmp/casino_data_schema.xsd"  # Temporary local path
xsd_content = "\n".join(row.value for row in xsd_df.collect())

# Save to local path
with open(local_xsd_path, "w") as f:
    f.write(xsd_content)

# Step 2: Load the XSD schema using xmlschema
xsd_schema = xmlschema.XMLSchema(local_xsd_path)

# Function to parse XSD and create a schema
def parse_xsd(xsd_file):
    tree = ET.parse(xsd_file)
    root = tree.getroot()
    schema = {}
    
    # Iterate through elements in XSD
    for complex_type in root.findall('.//xs:complexType', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
        type_name = complex_type.get('name')
        fields = []
        
        for sequence in complex_type.findall('.//xs:sequence', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
            for element in sequence.findall('xs:element', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
                field_name = element.get('name')
                field_type = element.get('type')
                # Map XSD types to Spark types
                spark_type = None
                if field_type == 'xs:string':
                    spark_type = StringType()
                elif field_type == 'xs:int':
                    spark_type = IntegerType()
                elif field_type == 'xs:decimal':
                    spark_type = DecimalType()
                elif field_type == 'xs:dateTime':
                    spark_type = TimestampType()
                
                if spark_type is not None:
                    fields.append(StructField(field_name, spark_type, True))
        
        schema[type_name] = StructType(fields)
    
    return schema

# Extract row tags (the high-level elements like users, games, transactions)
def extract_row_tags(xsd_path):
    # Parse the XSD file
    schema_tree = ET.parse(xsd_path)
    root = schema_tree.getroot()

    # Find all the top-level row tags
    row_tags = []
    for element in root.findall('.//xs:element', namespaces={'xs': 'http://www.w3.org/2001/XMLSchema'}):
        row_tags.append(element.attrib['name'])
    
    return row_tags

row_tags = extract_row_tags(local_xsd_path)

# Load the XML data
xml_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/casino_data.xml"  # Adjust to your XML file path

# Read the XML data at the top level (casino)
df = spark.read \
    .format("xml") \
    .option("rowTag", "casino") \
    .load(xml_path)

# Now, parse the XSD schema for field structures
schema = parse_xsd(local_xsd_path)

# Dictionary to store DataFrames for each high-level tag (users, games, transactions)
dataframes = {}

# Process each row tag (e.g., users, games, transactions)
for tag in row_tags:
    # Extract the respective DataFrame
    if tag == "users":
        users_df = df.select(explode("users.user").alias("user")).select("user.*")
        dataframes["users"] = users_df
    elif tag == "games":
        games_df = df.select(explode("games.game").alias("game")).select("game.*")
        dataframes["games"] = games_df
    elif tag == "transactions":
        transactions_df = df.select(explode("transactions.transaction").alias("transaction")).select("transaction.*")
        dataframes["transactions"] = transactions_df

# Display the DataFrames
for tag, df in dataframes.items():
    print(f"Data for {tag}:")
    df.show(truncate=False)

# Write each DataFrame to the Lakehouse as a Delta table
for tag, df in dataframes.items():
    table_name = f"{tag}_table"  # e.g., users_table, games_table, etc.
    

# Clean up the local XSD file if needed
os.remove(local_xsd_path)


StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 58, Finished, Available, Finished)

Data for users:
+-------+----------------------+-------------+-------------------+---------+-------+----------+
|balance|email                 |password_hash|registration_date  |status   |user_id|username  |
+-------+----------------------+-------------+-------------------+---------+-------+----------+
|250.75 |john.doe@example.com  |abc123xyz    |2022-01-15 10:30:00|active   |1      |john_doe  |
|800.0  |jane.smith@example.com|xyz456abc    |2021-05-22 14:45:00|suspended|2      |jane_smith|
+-------+----------------------+-------------+-------------------+---------+-------+----------+

Data for games:
+-------+-------+-------+----------------+-----------+---------+
|game_id|max_bet|min_bet|name            |provider   |type     |
+-------+-------+-------+----------------+-----------+---------+
|1      |100.0  |1.0    |Lucky Slots     |SpinTech   |slot     |
|2      |500.0  |5.0    |Blackjack Royale|CardMasters|blackjack|
+-------+-------+-------+----------------+-----------+---------+



In [50]:
%%sql
select * from user_id;

StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 53, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 0 fields>

In [21]:
import os

# File paths to check
xsd_path = "./builtin/casino_data_schema.xsd"
xml_path = "./builtin/Customer.csv"

# Check if files exist
if os.path.exists(xsd_path):
    print(f"XSD file {xsd_path} exists.")
else:
    print(f"XSD file {xsd_path} does not exist.")

if os.path.exists(xml_path):
    print(f"XML file {xml_path} exists.")
else:
    print(f"XML file {xml_path} does not exist.")


StatementMeta(, 75ddc290-a293-4c13-874c-badee122e0e8, 25, Finished, Available, Finished)

XSD file ./builtin/casino_data_schema.xsd exists.
XML file ./builtin/Customer.csv exists.


In [6]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Lakehouse CSV Reader") \
    .getOrCreate()

# Ensure this path is correct and accessible
csv_path = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/772c1423-20a6-4e59-84ee-d82652f1f1e8/Files/Customer.csv"  # Use a descriptive variable name for clarity

# Try loading the CSV file
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_path)

# Show the DataFrame
df.show()

StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 9, Finished, Available, Finished)

+-----------+------------------+----------------+------------+------------+--------------------+--------------+-------------+--------------------+-----------+-------------+-------------------+-------------------+-------------------+
|Customer_ID|              Name|        Password| Cell_Number|Phone_Number|               Email|         State|         City|             Address|Postal Code|Date of Birth|         CreditCard|   Registrationdate|Last logged in time|
+-----------+------------------+----------------+------------+------------+--------------------+--------------+-------------+--------------------+-----------+-------------+-------------------+-------------------+-------------------+
|          1|     Ingaberg Spir|yH6/7119!NfyiK}k|520 323 5886|404 567 0963|   ispir0@zimbio.com|       Arizona|       Tucson| 7836 Brown Junction|      85737|    9/30/1994|   3553975076398062|2013-09-02 07:12:04|2018-01-21 01:34:22|
|          2|      Morna Jewers|   gB1/PJw`ZhY\v|508 654 2980|202 52

In [31]:

# Import necessary modules
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import from_json, col, count
from pyspark.sql import functions as F

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Kafka Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

# Kafka configuration
kafka_bootstrap_servers = "158.220.124.0:9094"  # Replace with your Kafka broker address

# Define the schema for customer data
customer_schema = StructType() \
    .add("customer_id", IntegerType()) \
    .add("first_name", StringType()) \
    .add("last_name", StringType()) \
    .add("email", StringType()) \
    .add("phone_number", StringType()) \
    .add("passport_number", StringType()) \
    .add("nationality", StringType())

pnr_schema = StructType([
    StructField("pnr_id", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("flight_number", StringType(), True),
    StructField("departure_airport", StringType(), True),
    StructField("arrival_airport", StringType(), True),
    StructField("departure_date", TimestampType(), True),
    StructField("arrival_date", TimestampType(), True),
    StructField("ticket_price", FloatType(), True),
    StructField("seat_number", StringType(), True),
    StructField("booking_status", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("passport_number", StringType(), True)
])

# Read from Kafka
cus_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", "customers_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr('CAST(value AS STRING)') \
    .select(from_json(col('value'), customer_schema).alias('data')).select('data.*')

pnr_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", "pnr_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr('CAST(value AS STRING)') \
    .select(from_json(col('value'), pnr_schema).alias('data')).select('data.*')

# Add a timestamp column to each DataFrame
cus_df_with_timestamp = cus_df.withColumn("timestamp", F.current_timestamp())
pnr_df_with_timestamp = pnr_df.withColumn("timestamp", F.current_timestamp())

# Apply watermark on the DataFrames
cus_df_with_watermark = cus_df_with_timestamp.withWatermark("timestamp", "10 minutes")
pnr_df_with_watermark = pnr_df_with_timestamp.withWatermark("timestamp", "10 minutes")

# Write to Delta tables

# Write to Delta tables using saveAsTable

test_query = df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "Files/RealTime/test_checkpoint") \
    .start("test_bronze_table")  # New table name for testing

query2 = pnr_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "Files/RealTime/pnr_checkpoint") \
    .start("bronze_pnr_table2")  # Use start with table name

# Start the query
test_query.awaitTermination()


StatementMeta(, 75ddc290-a293-4c13-874c-badee122e0e8, 35, Finished, Available, Finished)

AnalysisException: [WRITE_STREAM_NOT_ALLOWED] `writeStream` can be called only on streaming Dataset/DataFrame.

In [34]:

#!/usr/bin/env python
# coding: utf-8

# ## FabricNotebook-To-ConnectKafka-Cluster
# 
# New notebook

# In[1]:


# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Kafka Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()


# In[2]:


# Kafka configuration
kafka_bootstrap_servers = "158.220.124.0:9094"  # Replace with your Kafka broker address
kafka_topic = "customers_data"                     # Replace with your Kafka topic  test_topic

# Read from Kafka
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "latest") \
  .option("failOnDataLoss", "false")\
  .load()

# Convert Kafka message key and value from binary to string
kafka_df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Define a function to print each consumed message
def process_batch(df, batch_id):
    # Print the batch id and each row
    print(f"Processing batch {batch_id}:")
    for row in df.collect():
        print(f"Key: {row['key']}, Value: {row['value']}")

# Output each message and write to console
query = kafka_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", "Files/RealTime/checkpoint") \
    .start()

query.awaitTermination()


StatementMeta(, 75ddc290-a293-4c13-874c-badee122e0e8, 38, Finished, Cancelled, Cancelled)

Processing batch 1:
Key: None, Value: {"customer_id": 1, "first_name": "Mark", "last_name": "Moreno", "email": "jasminesingh@example.net", "phone_number": "511.590.9332x024", "passport_number": "I04360617", "nationality": "Greenland"}
Key: None, Value: {"customer_id": 2, "first_name": "Rachel", "last_name": "Herrera", "email": "matthewsjoshua@example.org", "phone_number": "+1-541-511-3413", "passport_number": "B88331200", "nationality": "Kuwait"}
Key: None, Value: {"customer_id": 3, "first_name": "Abigail", "last_name": "Mcbride", "email": "mark67@example.org", "phone_number": "272-943-6872x110", "passport_number": "945878024", "nationality": "Afghanistan"}
Key: None, Value: {"customer_id": 4, "first_name": "Katie", "last_name": "Wise", "email": "petersonrichard@example.net", "phone_number": "595.968.2058x811", "passport_number": "P79287707", "nationality": "Algeria"}
Key: None, Value: {"customer_id": 5, "first_name": "Melissa", "last_name": "Nelson", "email": "josephhill@example.com",

In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Create Delta Table") \
    .getOrCreate()

# Sample data for creating the Delta table
data = [(1, "John Doe", "2024-10-09 10:00:00"),
        (2, "Jane Smith", "2024-10-09 10:05:00")]

columns = ["id", "name", "timestamp"]

# Create a DataFrame
df = spark.createDataFrame(data, columns)
df.show()
# Write the DataFrame to a Delta table
df.write.mode("overwrite").format("delta").saveAsTable("test_Data")
  # Specify your Delta Lake path

# Register the Delta table in the catalog (optional)
#spark.sql("CREATE TABLE bronze_customer_table USING DELTA LOCATION '/path/to/delta/bronze_customer_table'")


StatementMeta(, d0b77ee1-0ffd-4bb7-9a93-8156c8bb2303, 5, Finished, Available, Finished)

+---+----------+-------------------+
| id|      name|          timestamp|
+---+----------+-------------------+
|  1|  John Doe|2024-10-09 10:00:00|
|  2|Jane Smith|2024-10-09 10:05:00|
+---+----------+-------------------+

